# Predicting the Norwegian Parliament Election 2025 Prediction

* A model using bayesian statistics to predict the outcome of the Parliement election.

### Notes
* 169 mandates, 150 direkte, 19 utgjevningsmandater, en fra hvert valg distrikt

In [34]:
import numpy as np
import pandas as pd
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
import re
pd.set_option('display.max_rows', None)

### Constants

In [14]:
valgdistrikt=np.array(["Østfold", "Akershus", "Oslo", "Hedemark", "Oppland", "Buskerud", "Vestfold","Telemark", "Aust-Agder", "Vest-Agder", "Rogaland","Hordaland","Sogn og Fjordane","Møre og Romsdal","Sør-Trøndelag","Nord-Trøndelag","Nordland","Troms", "Finnmark"])

In [16]:
partier= np.array(['Ap', 'Høyre', 'Frp', 'SV', 'Sp', 'KrF', 'Venstre', 'MDG', 'Rødt', 'Andre'])

In [98]:
urls=["https://www.pollofpolls.no/?cmd=Stortinget&fylke=1",
      "https://www.pollofpolls.no/?cmd=Stortinget&fylke=2",
      "https://www.pollofpolls.no/?cmd=Stortinget&fylke=3",
      "https://www.pollofpolls.no/?cmd=Stortinget&fylke=4",
      "https://www.pollofpolls.no/?cmd=Stortinget&fylke=5",
      "https://www.pollofpolls.no/?cmd=Stortinget&fylke=6",
      "https://www.pollofpolls.no/?cmd=Stortinget&fylke=7",
      "https://www.pollofpolls.no/?cmd=Stortinget&fylke=8",
      "https://www.pollofpolls.no/?cmd=Stortinget&fylke=9",
      "https://www.pollofpolls.no/?cmd=Stortinget&fylke=10",
      "https://www.pollofpolls.no/?cmd=Stortinget&fylke=11",
      "https://www.pollofpolls.no/?cmd=Stortinget&fylke=12", #hordaland
      "https://www.pollofpolls.no/?cmd=Stortinget&fylke=14", #sogn
      "https://www.pollofpolls.no/?cmd=Stortinget&fylke=15",
      "https://www.pollofpolls.no/?cmd=Stortinget&fylke=16",
      "https://www.pollofpolls.no/?cmd=Stortinget&fylke=17",
      "https://www.pollofpolls.no/?cmd=Stortinget&fylke=18",
      "https://www.pollofpolls.no/?cmd=Stortinget&fylke=19",
      "https://www.pollofpolls.no/?cmd=Stortinget&fylke=20",
      ]

### Data scraping
Extracting the local polling data in each the 19 election districts from https://www.pollofpolls.no. For each Election district the polling data concists of the distribtuion of votes (%) between the parties Ap, Høyre, Frp, SV, Sp, KrF, Venstre, MDG, Rødt, Andre. 

In [99]:
def extract_local_polling_data(url_list):
    valgdistrikt=np.array(["Østfold", "Akershus", "Oslo", "Hedemark", "Oppland", "Buskerud", "Vestfold","Telemark", "Aust-Agder", "Vest-Agder", "Rogaland","Hordaland","Sogn og Fjordane","Møre og Romsdal","Sør-Trøndelag","Nord-Trøndelag","Nordland","Troms", "Finnmark"])   
    partier= np.array(['Ap', 'Høyre', 'Frp', 'SV', 'Sp', 'KrF', 'Venstre', 'MDG', 'Rødt', 'Andre'])
    df_new = pd.DataFrame({"Valgdistrikt": np.repeat(valgdistrikt, 10, axis=0), "Partier": np.tile(partier,19),"Poll-dato":np.nan})

    prosent_lister=np.array([])
    dato_lister=np.array([])

    for url in urls:
        page = urlopen(url)
        html_bytes = page.read()
        html = html_bytes.decode("utf-8")
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.find('table') #find all tables
        rows = table.find_all('tr') # find each row
        prosenter=[]
        for row in rows:
            cells = row.find_all(['th', 'td'])  # Find both header and data cells
            cell_texts = [cell.get_text(strip=True) for cell in cells]

            if cell_texts[0]=="Siste lokale målings":
                for i in range(1,11): #number of parties
                    tekst = cell_texts[i]
                    index = tekst.find(' (')
                    tall_string_format = tekst[0:index]
                    tall_string_med_komma = tall_string_format.replace(',','.')
                    tall = float(tall_string_med_komma)
                    prosenter.append(tall)
        prosent_lister=np.append(prosent_lister,prosenter)
        # add date to poll
        for li in soup.find_all('li'):
            sup_tag = li.find('sup')
            if sup_tag and sup_tag.text=='s':
                text = li.get_text()
                index1=text.find('(')
                index2=text.find(')')
                date=text[index1+1:index2]
                break
        dato_lister=np.append(dato_lister,[date for i in range(0,10)])


    df_new['Prosent-oppsluttning'] = prosent_lister
    df_new['Poll-dato'] = dato_lister

    return df_new
                

In [100]:
df_poll=extract_local_polling_data(urls)

In [101]:
df_poll

,Valgdistrikt,Partier,Poll-dato,Prosent-oppsluttning
0,Østfold,Ap,10/9-2021,27.5
1,Østfold,Høyre,10/9-2021,17.8
2,Østfold,Frp,10/9-2021,15.9
3,Østfold,SV,10/9-2021,7.7
4,Østfold,Sp,10/9-2021,12.9
5,Østfold,KrF,10/9-2021,2.8
6,Østfold,Venstre,10/9-2021,3.1
7,Østfold,MDG,10/9-2021,4.4
8,Østfold,Rødt,10/9-2021,5.0
9,Østfold,Andre,10/9-2021,3.1


### Load Election district Population and Mandate distribution

In [17]:
col_names = ["Valgdistrikt","Mandater","Befolkningstall"]
df = pd.read_csv('./data/mandate-distribution.csv',header=None,names=col_names) 
df 

,Valgdistrikt,Mandater,Befolkningstall
0,Østfold,9,312152
1,Akershus,20,728803
2,Oslo,20,717710
3,Hedemark,7,202048
4,Oppland,6,174256
5,Buskerud,8,269819
6,Vestfold,7,256432
7,Telemark,6,177093
8,Aust-Agder,4,122968
9,Vest-Agder,6,196882
